# Lecture 9: Methods using KKT conditions

# Live exercise
Work in pairs and use Internet to find anything related to
* Sequential Quadratic Programming (SQP) and
* Augmented Lagrangian method

Time: 10 minutes

## Sequential Quadratic Programming (SQP)

**Idea is to generate a sequence of quadratic optimization problems whose solutions approach the solution of the original problem**

Let us consider problem
$$
\min f(x)\\
\text{s.t. }h_k(x) = 0\text{ for all }k=1,\ldots,K,
$$
where the the objective function and the equality constraints are twice differentiable. 

Inequality constraints can be handled e.g. by using active set methods. See e.g. https://optimization.mccormick.northwestern.edu/index.php/Sequential_quadratic_programming

Note that constraints can be nonlinear.

Because we know that the optimal solution of this problem satisfies the KKT conditions, we know that
$$
\left\{\begin{array}{l}
\nabla_xL(x,\mu)=\nabla f(x) + \mu\nabla h(x) = 0\\
h(x) = 0
\end{array}\right.
$$
Let us assume that we have a current esimation for the solution of the equality constraints $(x^k,\mu^k)$, then according to the Newton's method for root finding (see e.g., https://en.wikipedia.org/wiki/Newton%27s_method ), we have another solution $(x^k,\mu^k)^T+(p,v)^T$ of the problem by solving system of equations
$$
\nabla^2_{xx}L(x,\mu)\left[\begin{align}p^T\\v^T\end{align}\right] = -\nabla_x L(x,\mu).
$$
This can be written as
$$
\left[
\begin{array}{cc}
\nabla^2_{xx}L(x^k,\mu^k)&\nabla h(x^k)\\
\nabla h(x^k)^T & 0
\end{array}
\right]
\left[\begin{array}{c}p^T\\v^T\end{array}\right] =
\left[
\begin{array}{c}
-\nabla_x L(x^k,\mu^k)\\
-h(x^k)^T
\end{array}
\right].
$$

However, the above is just the solution of the quadratic problem with equality constraints
$$
\min \frac12 p^T\nabla^2_{xx}L(x^k,\mu^k)p+\nabla_xL(x^k,\mu^k)^Tp\\
\text{s.t. }h_j(x^k) + \nabla h_j(x^k)^Tp = 0. 
$$

## Intuitive interpretation

We are approximating the Lagrange function quadratically around the current solution and the constraints are approximated linearly. SQP methods are also referred to *projected Lagrangian methods*

**Another view point to building the approximation**: Assume that we have a current solution candidate $(x^k,\mu^k)$. Using Taylor's series for the constraints ($d = x^*-x^k$) and including only the first order term we get

$h_i(x^k+d)\approx h_i(x^k) + \nabla h_i(x^k)^Td$

Since, $h_i(x^*)=0$ for all $i$ we have

$\nabla h(x^k)d = -h(x^k)$

For approximating the Lagrangian function, we use up to second order terms and get

$L(x^k+d,\mu^*) \approx L(x^k,\mu^*) + d^T\nabla_x L(x^k,\mu^*) + \frac{1}{2}d^T\nabla_{xx}^2L(x^k,\mu^*)d$

When combining both approximations, we get a quadratic subproblem

$$
\underset{d}{\min}d^T\nabla_x L(x^k,\mu^k) + \frac{1}{2}d^T \nabla_{xx}^2L(x^k,\mu^k)d\\
\text{s.t. } \nabla h(x^k)d = -h(x^k)
$$

It can be shown (under some assumptions) that solutions of the quadratic subproblems approach $x^*$ annd Lagrange multipliers approach $\mu^*$.

Note that we can either use the exact Hessian of the Lagrange function (requires second derivatives) or some approximation of it (compare Newton's method vs. quasi-Newton ideas). 

## Implementation

Define an optimization problem, where
* $f(x) = \|x\|^2 = \sum_{i=1}^n x_i^2$
* $h(x) = \sum_{i=1}^nx_i-n$

What is $x^*$?

In [ ]:
def f_constrained(x):
    return sum([i**2 for i in x]),[],[sum(x)-len(x)]
#    return sum([i**2 for i in x]),[],[sum(x)-len(x),x[0]**2+x[1]-2]


In [ ]:
print(f_constrained([1,0,1]))
print(f_constrained([1,2,3,4]))

In [ ]:
import numpy as np
import ad


#if k=0, returns the gradient of lagrangian, if k=1, returns the hessian
def diff_L(f,x,m,k):
    #Define the lagrangian for given m and f
    L = lambda x_: f(x_)[0] + (np.matrix(f(x_)[2])*np.matrix(m).transpose())[0,0]
    return ad.gh(L)[k](x)
#Returns the gradients of the equality constraints
def grad_h(f,x):
    return  [ad.gh(lambda y:
                   f(y)[2][i])[0](x) for i in range(len(f(x)[2]))] 

#Solves the quadratic problem inside the SQP method
def solve_QP(f,x,m):
    left_side_first_row = np.concatenate((\
    np.matrix(diff_L(f,x,m,1)),\
    np.matrix(grad_h(f,x)).transpose()),axis=1)
    left_side_second_row = np.concatenate((\
    np.matrix(grad_h(f,x)),\
    np.matrix(np.zeros((len(f(x)[2]),len(f(x)[2]))))),axis=1)
    right_hand_side = np.concatenate((\
    -1*np.matrix(diff_L(f,x,m,0)).transpose(),
    -np.matrix(f(x)[2]).transpose()),axis = 0)
    left_hand_side = np.concatenate((\
                                    left_side_first_row,\
                                    left_side_second_row),axis = 0)
    temp = np.linalg.solve(left_hand_side,right_hand_side)
    return temp[:len(x)],temp[len(x):]
    
    

def SQP(f,start,precision):
    x = start
    m = np.ones(len(f(x)[2]))
    f_old = float('inf')
    f_new = f(x)[0]
    while abs(f_old-f_new)>precision:
        print(x)
        f_old = f_new
        (p,v) = solve_QP(f,x,m)
        x = x+np.array(p.transpose())[0]
        m = m+v
        f_new = f(x)[0]
    return x

In [ ]:
x0 = [2000.0,1000.0,3000.0,5000.0,6000.0]
SQP(f_constrained,x0,0.0001)

## Lagrangian methods -- "The original method of multipliers"

Let us again consider problem
$$
\min f(x)\\
\text{s.t. }h_k(x) = 0\text{ for all }k=1,\ldots,K,
$$
where the objective function and the equality constraints are twice differentiable. Inequality constaints can be handled e.g. by first converting them into equality constraints which increases the number of variables.

We know that if $x^*$ is optimal solution, then $\nabla_x L(x^*,\mu^*) = \nabla f(x^*) + \sum_{k=1}^K\mu^*_k \nabla h_k(x^*) = 0$ 
(<i>necessary condition</i>). 

<b>However</b>, if we only know that for $x^*$ it holds that $\nabla_x L(x^*,\mu^*) = 0$, we can't be sure that $x^*$ is a local minimizer. (Why???)

Since the Hessian $\nabla^2_{xx}L(x^*,\mu^*)$ may be indefinitive, it is not sufficient to just minimize the Langrangian function $L(x,\mu)$ in order to minimize $f(x)$ with respect to the equality constraints $h_k(x)=0$.

<i>Solution: Improve Lagrangian function</i>

Define augmented Lagrangian function
$$
L_c(x,\mu) = f(x)+\mu h(x)+\frac12c\|h(x)\|^2.
$$
Above $c\in \mathbb R$ is a penalty parameter and $\mu \in \mathbb R^K$ is a multiplier.

Now, we have
$$
\nabla^2_{xx}L_c(x^*,\mu^*) = \nabla^2_{xx}L(x^*,\mu^*) + c\nabla h(x^*)^T\nabla h(x^*)
$$
and it can be shown that for $c>\hat{c}$ the Hessian of the augmented Lagrangian is positive definite (<i>sufficient condition for optimality</i>).

Let us consider sequence of optimization problems
$$
\min_{x\in\mathbb R^n} f(x)+\mu_k h(x)+\frac{1}{2}c_k\|h(x)\|^2,
$$
where $c_{k+1}>c_k$ for $k=1,2,\ldots$.

Now, if $\mu_k=0$ for all $k=1,2,\ldots$, then we have a penalty function method, which solves the problem when $c_k\to \infty$.

However, it can be shown, that if we set $\mu_0$ randomly and keep on updating
$\mu_{k+1} = \mu_k+c_kh(x_k)$, then we can show that there exists $C>0$ such that of $c_k>C$, then the optimal solution of the augmented Langrangian solves the original problem!

### Example

Let us have optimization problem
$$
\min x_1^2+x_2^2\\
\text{s.t. }x_1+x_2-1=0.
$$

Now, the minimization of the augmented Lagrangian becomes
$$
\min_{x\in\mathbb R^n} x_1^2+x_2^2+\mu_k(x_1+x_2-1)+\frac12c_k(x_1+x_2-1)^2.\\
$$

In [ ]:
def f_constrained2(x):
    return sum([i**2 for i in x]),[],[sum(x)-1]

In [ ]:
def augmented_langrangian(f,x,mu,c):
    second_term = float(numpy.matrix(mu)*numpy.matrix(f(x)[2]).transpose())
    third_term = 0.5*c*numpy.linalg.norm(f(x)[2])**2
    return f(x)[0]+second_term+third_term

In [ ]:
from scipy.optimize import minimize
import numpy
def augmented_langrangian_method(f,start,mu0,c0):
    x_old = [float('inf')]*2
    x_new = start
    mu = mu0
    c = c0
    while numpy.linalg.norm(f(x_new)[2])>0.00001:
        res = minimize(lambda x:augmented_langrangian(f,x,mu,c),x_new)
        x_old = x_new
        mu = float(mu+numpy.matrix(c)*numpy.matrix(f(res.x)[2]).transpose())
        x_new = res.x
        c = 2*c
    return x_new,c

In [ ]:
from scipy.optimize import minimize
import numpy
def penalty_function_method(f,start,c0):
    x_old = float('inf')
    x_new = start
    c = c0
    while numpy.linalg.norm(f(x_new)[2])>0.00001:
        res = minimize(lambda x:augmented_langrangian(f,x,0,c),x_new)
        x_old = x_new
        x_new = res.x
        c = 2*c
    return x_new,c

In [ ]:
x0 =[10,-5]
augmented_langrangian_method(f_constrained2,x0,1,1)

In [ ]:
penalty_function_method(f_constrained2,x0,1)

In [ ]:
SQP(f_constrained2,x0,0.00001)

## What is going on in here?

The above is a simplified representation of the augmented Lagrangian method. For example, one can use exact second derivatives for calculating $\nabla^2_{xx}L(x^*,\mu^*)$ to obtain better convergence but, also, one can approximate it by utilizing ideas from quasi-Newton methods in order to not requiring second derivatives. Efficient implementation of this (and other methods) for practical problems is not completely trivial, unfortunately. If you want to read details, please see e.g., http://www.mit.edu/~dimitrib/Constrained-Opt.pdf.